In [1]:
import os
import sys
import numpy as np
import pandas as pd
import astropy.units as u
import astropy.constants as const

import matplotlib
import matplotlib.pyplot as plt

from cube.visualization import standard
plt.style.use(standard)

amapola_data_file = '../../data/amapola.txt'
amapola_data = pd.read_csv(amapola_data_file, delimiter='\t')
amapola_data['Date'] = pd.to_datetime(amapola_data['Date'])
amapola_data['polpercentage'] = 100 * np.sqrt(amapola_data['Q']**2 + amapola_data['U']**2) / amapola_data['I']
amapola_data['p'] =  amapola_data['P'] / amapola_data['I']

In [2]:
(const.c / ( 1.1 * u.mm)).to(u.GHz), (const.c / ( 233 * u.GHz)).to(u.mm)

(<Quantity 272.53859818 GHz>, <Quantity 1.28666291 mm>)

In [3]:
band_7 = amapola_data.loc[amapola_data['Freq'] == 233].copy()
band_7.head()

,Src,Freq,EL,I,Q,U,V,eI,eQ,eU,eV,Date,File,P,eP_lower,eP_upper,EVPA,eEVPA,polpercentage,p
2916,J2258-2758,233.0,42.4,0.5756,0.01850,-0.033390,-0.007710,0.010440,0.005804,0.006799,0.001444,2014-01-27 23:11:51,uid___A002_X79b541_X194-RB_06-Flux.log,0.03817,0.03203,0.045433,-0.5325,0.07928,6.631781,0.066313
2917,J2357-5311,233.0,47.4,0.5972,-0.02298,-0.006617,0.006573,0.007969,0.004529,0.001625,0.003112,2014-01-27 23:11:51,uid___A002_X79b541_X194-RB_06-Flux.log,0.02391,0.02078,0.027491,-1.4310,0.04186,4.004304,0.040037
2918,J0319+4130,233.0,24.9,9.5750,-0.06596,-0.058220,-0.007919,0.172600,0.014030,0.097290,0.004538,2014-01-27 23:11:51,uid___A002_X79b541_X194-RB_06-Flux.log,0.08798,0.05656,0.146311,-1.2090,0.41790,0.918840,0.009189
2919,J0423-0120,233.0,59.2,1.6280,0.03533,-0.022690,-0.016890,0.037190,0.008616,0.000637,0.004463,2014-01-27 23:11:51,uid___A002_X79b541_X194-RB_06-Flux.log,0.04199,0.03675,0.047934,-0.2854,0.05580,2.579154,0.025792
2920,J0237+2848,233.0,37.8,1.4010,-0.05854,0.006465,-0.047780,0.019640,0.004185,0.007910,0.004367,2014-01-27 23:11:51,uid___A002_X79b541_X194-RB_06-Flux.log,0.05889,0.05197,0.066683,1.5160,0.06686,4.203848,0.042034


In [4]:
for source in band_7['Src'].unique():
    
    this_source_only = band_7.loc[band_7['Src'] == source].copy()
    plot_obj, sp_obj = plt.subplots(2, 1, figsize=(8, 5), sharex=True, dpi=200) # gridspec_kw={'hspace': 0, 'wspace': 0}
    sp_obj = sp_obj.ravel()

    sp_obj[0].set_title(f"{source}")

    # Total Intensity
    sp_obj[0].plot(this_source_only['Date'], this_source_only['I'], '.', color='firebrick')
    sp_obj[0].errorbar(this_source_only['Date'], this_source_only['I'], this_source_only['eI'], ls='none', color='firebrick')
    sp_obj[0].axhline(np.mean(this_source_only['I']),  ls='--', label=f"mean = {np.mean(this_source_only['I']):0.3f}, std={np.std(this_source_only['I']):0.3f}", color='firebrick')
    sp_obj[0].axhspan(np.mean(this_source_only['I']) - np.std(this_source_only['I']), np.mean(this_source_only['I']) + np.std(this_source_only['I']), alpha=0.12, color='red', edgecolor=None)
    sp_obj[0].set_ylim((0.0, 10))
    sp_obj[0].set_ylabel("Stokes I [Jy]")
    sp_obj[0].legend(fancybox=False, loc='upper right', handletextpad=0.7, frameon=False)

    # Polarization Percentage
    sp_obj[1].plot(this_source_only['Date'], this_source_only['polpercentage'], '.')
    sp_obj[1].axhline(np.mean(this_source_only['polpercentage']),  ls='--', label=f"mean = {np.mean(this_source_only['polpercentage']):0.3f}, std={np.std(this_source_only['polpercentage']):0.3f}", color='darkorchid')
    sp_obj[1].axhspan(np.mean(this_source_only['polpercentage']) - np.std(this_source_only['polpercentage']), np.mean(this_source_only['polpercentage']) + np.std(this_source_only['polpercentage']), alpha=0.12, color='red', edgecolor=None)
    sp_obj[1].set_ylim((0.0, 10))
    sp_obj[1].set_ylabel("Pol. Percentage [\%]")
    sp_obj[1].legend(fancybox=False, loc='upper right', handletextpad=0.7, frameon=False)

    plot_obj.tight_layout()
    plot_obj.savefig(f'../individual_plots/{source.strip()}.png', dpi=400);
    plt.close('all')

In [ ]:
# pruned_sources = []
# observations_per_source = []
# for source in band_7['Src'].unique():
#     one_source_only = band_7.loc[band_7['Src'] == source].copy()
#     if one_source_only.shape[0] > 5 :
#         if np.mean(one_source_only['I']) > 1:
#             pruned_sources.append(source)
#             observations_per_source.append(one_source_only.shape[0])
# len(observations_per_source)

In [ ]:
# polpercs = []
# for source in pruned_sources:
#     this_source_only = band_7.loc[band_7['Src'] == source].copy()
#     polpercs.append(np.mean(this_source_only['polpercentage']))
# print(np.max(polpercs))